In [ ]:
%cd ../..
%pip install .
# offset = Uniform(-1, 1) * Range(90, 180) deg

# ego = Car on drivableRoad,
#         facing offset relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# otherCar = Car on visible road,
#             facing Range(-15, 15) deg relative to roadDirection

# require (distance from ego to otherCar) < 10

In [ ]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'car'
) as cars

select *
from cars as otherCar
join Cameras as cam on t1.cameraId = Cameras.id
where
  TODO: ego facing relative to road direction????
  DISTANCE(cam.egoTranslation, otherCar.trajCentroids, cam.timestamp) < 50 AND
  viewAngle(otherCar.traj, cam.egoHeading, cam.ego_Translation, cam.timestamp) < 135 AND
  contained(otherCar.traj, road_segment_type("visibleRoad"), cam.timestamp) and
  facingRelative(otherCar.heading, road_direction(valueAtTimestamp(otherCar.trajCentroids, cam.timestamp)), cam.timestamp) >= -15 AND
  facingRelative(otherCar.heading, road_direction(valueAtTimestamp(otherCar.trajCentroids, cam.timestamp)), cam.timestamp) < 15 AND
  DISTANCE(cam.egoTranslation, otherCar.centroid, cam.timestamp) < 10 AND
"""

In [ ]:
### Prepare the world for queries ###
from apperception.new_world import *
# directly ingest the mini dataset and boston roadnetwork if needed
# from benchmarks.ingest_data import ingest_data
# ingest_data()

# World.db.reset()
name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F
world = world.filter(lambda obj: obj.object_type == 'vehicle.cars')
def pred(otherCar, camera):
    return (
        # TODO: ego facing `offset` relative to road direction??
        F.distance(cam.egoTranslation, otherCar.trajCentroids, cam.timestamp) < 50 and
        F.viewAngle(otherCar.trajCentroids, cam.egoHeading, cam.egoTranslation, cam.timestamp) < 135 and
        F.contained(otherCar.trajCentroids, road_segment_type("visibleRoad"), cam.timestamp) and
        F.facingRelative(otherCar.headings, road_direction(valueAtTimestamp(otherCar.trajCentroids, cam.timestamp)), cam.timestamp) >= -15 and
        F.facingRelative(otherCar.headings, road_direction(valueAtTimestamp(otherCar.trajCentroids, cam.timestamp)), cam.timestamp) < 15 and
        F.distance(cam.egoTranslation, otherCar.centroid, cam.timestamp) < 10 
    )
world = world.filter(pred)